In [ ]:
%matplotlib                    # Pyplot 그래프 새창에서 열기 옵션

In [ ]:
%matplotlib inline             # Pyplot 그래프 새창에서 열기 '취소' 옵션

In [ ]:
pd.set_option('display.max_rows',    100)      # DataFrame 데이터 확인 시 최대 표시 행 수
pd.set_option('display.max_columns', 100)      # DataFrame 데이터 확인 시 최대 표시 열 수

## 라이브러리 import

In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import pickle
from sklearn import svm
import joblib
from sklearn.model_selection import GridSearchCV

## 데이터 불러오기

In [ ]:
Fold = 5

# k-fold 학습/검증 데이터
for i in range(Fold):
    
    path1 = './K_FoldData/Training_Fold%d'%(i+1)
    path2 = './K_FoldData/Validation_Fold%d'%(i+1)
    c1 = 'Training_Fold%d   = np.array(pd.read_csv(path1, sep=",", header=None))'%(i+1)
    c2 = 'Validation_Fold%d = np.array(pd.read_csv(path2, sep=",", header=None))'%(i+1)
    exec(c1)
    exec(c2)

# K-fold 학습/검증 레이블
TrainingFold_Label   = np.array(pd.read_csv('./K_FoldData/TrainingFold_Label'  , sep=",", header=None).T.squeeze())
ValidationFold_Label = np.array(pd.read_csv('./K_FoldData/ValidationFold_Label', sep=",", header=None).T.squeeze())
    
    
# 전체 학습용 데이터
Training_All       = np.array(pd.read_csv('./K_FoldData/Training_All', sep = ",", header = None))
Training_All_Label = np.array(pd.read_csv('./K_FoldData/Training_All_Label', sep = ",", header = None).T.squeeze())


# 평가 데이터
Test_Data  = np.array(pd.read_csv('./K_FoldData/Test_Data' , sep=",", header=None))
Test_Label = np.array(pd.read_csv('./K_FoldData/Test_Label', sep=",", header=None))

## Grid Search 기반 Hyperparameter 별 성능 확인

#### 1. 반복문 활용

In [ ]:
Fold = 5

# 비교할 하이퍼파라미터들 리스트 형태로 만들기
param_kernel = ['linear', 'poly', 'rbf', 'sigmoid'] # hyperplane 모양 rbf 고차원 분포에 유효
param_C      = [0.01, 0.1, 1, 10, 100]              # 이상치 허용 정도 c 값이 작아질 수로 이상치를 크게 허용 정제된 데이터의 경우 c파라미터 높게
param_gamma  = [0.01, 0.1, 1, 10, 100]              # hyperplane 곡률(얼마나 가프르게 만들어지냐 높으면 급한느낌) 선형함수에 적용x

# 파라미터별 정확도 기록할 변수 설정
Accuracy_df = pd.DataFrame(np.zeros(shape=(len(param_kernel)*len(param_C)*len(param_gamma),4)),
                           columns=['kernel', 'C', 'gamma', 'Accuracy'])
cnt = 0

# Grid Search 코드
for Kernel in param_kernel:
    
    for c in param_C:
        
        for Gamma in param_gamma:
            
            ValidScore = 0 # 모델 바뀔 때마다 검증정확도 변수 초기화
            
            for i in range(Fold):
                c1 = 'Training_CurrentFold = Training_Fold%d'%(i+1)
                exec(c1)
                c2 = 'Validation_CurrentFold = Validation_Fold%d'%(i+1)
                exec(c2)
                
                tempsvmModel   = svm.SVC(kernel=Kernel,
                                         C=c,
                                         gamma=Gamma).fit(Training_CurrentFold, TrainingFold_Label) # 학습
                tempValidScore = tempsvmModel.score(Validation_CurrentFold, ValidationFold_Label)   # 검증
                
                ValidScore += tempValidScore # Fold 별 검증 정확도 합산 
                
            Acc = ValidScore/Fold # 평균 검증 정확도
            Accuracy_df.iloc[cnt, :] = [Kernel, c, Gamma, Acc] # 모델의 하이퍼파라미터
            cnt += 1

In [ ]:
Accuracy_df_sorted = Accuracy_df.sort_values(by = ['Accuracy'], ascending = False) # 성능 확인
Accuracy_df_sorted

#### 2. sklearn.model_selection.GridSearchCV 함수 활용 (K-fold Cross Validation 내장)

In [ ]:
# 비교할 하이퍼파라미터 사전(Dictionary) 형태로 만들기
GridParams = {
    'kernel' : ['linear', 'poly', 'rbf', 'sigmoid'],
    'C' : [0.01, 0.1, 1, 10, 100],
    'gamma' : [0.01, 0.1, 1, 10, 100]
}

In [ ]:
tempsvmModel = svm.SVC()
GS_model = GridSearchCV(tempsvmModel, GridParams, cv = 5) #꼭 약수 일 필요는 없다

GS_model.fit(Training_All, Training_All_Label) # 학습

In [ ]:
print('Total Combination of Hyperparameters :', len(GS_model.cv_results_.get('params')))

In [ ]:
print('Best Hyperparameters : ', GS_model.best_params_) # 최고 검증정확도 하이퍼파라미터
print('Best Score : {}%'.format(round(GS_model.best_score_*100, 2)) )

# 모델 평가
# Test_Score = GS_model.score(Test_Data, Test_Label)
# print('Best Test Score : {}%'.format(round(Test_Score*100, 2)) )